In [52]:
from SALib.sample import morris
from SALib.analyze import morris
import numpy as np
import pandas as pd
import re, os
import time
import matplotlib.pyplot as plt

Set `projdir` on your system, then everything else is defined relative to that. `pwd` will just give the directory that this Jupyter notebook is housed in.

In [53]:
projdir = os.getcwd()
datdir = projdir + "/../ciam-code/output/MonteCarlo"
plotdir = projdir + "/../ciam-code/figures"

If the plot directory `plotdir` doesn't exist, make it.

In [54]:
if not os.path.exists(plotdir):
    os.makedirs(plotdir)
print("Will save plots to ",plotdir)

Will save plots to  /Users/aewsma/codes/CIAM_uncertainty_propagation/work_uncertainty_propagation/../ciam-code/figures


BRICK parameters are the same for all scenarios because they are calibrated in hindcast.

In [62]:
brickdir = "https://zenodo.org/record/6461560/files/parameters_subsample_sneasybrick_20M_19-02-2022.csv"
dfPB = pd.read_csv(brickdir)

SSP-RCP scenarios and a dictionary to hold the sensitivity results from Method of Morris:

In [142]:
surge_option = 1
scenarios = [(1,26),(2,45),(4,60),(5,85)]
dfSi = {scen : None for scen in scenarios}

Directories where all the BRICK-CIAM results are, for all the different SSP-RCP scenarios.

In [134]:
for (ssp, rcp) in scenarios:
    bothdir = datdir + "/SSP"+str(ssp)+"_BRICK"+str(rcp)+"/SSP"+str(ssp)+"_BRICK"+str(rcp)+"_surge"+str(surge_option)+"_varySLR_varyCIAM/CIAM MC1000/PostProcessing"
    dfSC = pd.read_csv(bothdir+"/globalnpv_SSP"+str(ssp)+"_BRICK"+str(rcp)+"_surge"+str(surge_option)+"_varySLR_varyCIAM.csv")
    dfSC = dfSC.join(pd.read_csv(bothdir+"/regionnpv_SSP"+str(ssp)+"_BRICK"+str(rcp)+"_surge"+str(surge_option)+"_varySLR_varyCIAM.csv"))
    dfPC = pd.read_csv(bothdir+"/trials_SSP"+str(ssp)+"_BRICK"+str(rcp)+"_surge"+str(surge_option)+"_varySLR_varyCIAM.csv",
                       names = ["movefactor","dvbm","vslel","vslmult","wvel","wvpdl"], header=0)

    tmp = dfPB.iloc[dfSC.brickEnsInd]
    tmp.reset_index(inplace=True, drop=True)
    dfP = pd.concat([tmp,dfPC], axis=1)
        
    n_ensemble, n_parameter = dfP.shape
    mins = list(dfP.min())
    maxs = list(dfP.max())
    bounds = [[mins[i],maxs[i]] for i in range(n_parameter)]

    problem = {"num_vars" : n_parameter,
               "names" : list(dfP.columns),
               "bounds" : bounds}

    X = np.array(dfP)[:986,:]
    Y = np.array(dfSC.npv)[:986]
    dfSi[(ssp,rcp)] = pd.DataFrame(SALib.analyze.morris.analyze(problem, X, Y, print_to_console=False))
    

In [141]:
dfPC.head()

,movefactor,dvbm,vslel,vslmult,wvel,wvpdl
0,2.737932,2.864736,0.423844,203.400648,1.984252,0.401850
1,1.146398,4.689004,0.460670,172.968618,1.024534,0.529294
2,1.210031,5.899834,0.315121,170.584669,0.940749,0.587431
3,1.518711,4.077409,0.294079,126.846041,1.413168,0.545942
4,2.410799,10.744820,0.207871,110.392481,1.155614,0.514876


* dvbm = FUND value of OECD dryland per Darwin et al 1995 converted from $1995 ($2010M per sqkm) (5.376)
* wvel = income elasticity of wetland value (1.16) (Brander et al, 2006)
* movefactor = Cost to relocate mobile capital as a fraction of asset value (0.1)
* vslel = Elasticity of vsl (0.5) (only used for endogenous calculation of vsl)
* vslmult = multiplier on USA GDP (216)(only used for endogenous calculation of vsl)
* wvpdl = Population density elasticity of wetland value (0.47) (Brander et al, 2

In [140]:
for scen in scenarios:
    print(dfSi[scen].sort_values(by="mu_star", ascending=False)[["names","mu_star"]][:10])

                       names       mu_star
52                      dvbm  20865.925264
55                      wvel   6897.025572
51                movefactor   5809.152771
41              antarctic_nu   5579.200261
1              sd_ocean_heat   5564.707463
45  antarctic_runoff_height0   5388.506925
13                alpha0_CO2   5221.768109
56                     wvpdl   5184.026472
49          antarctic_lambda   5163.351064
19              greenland_v0   5106.884970
            names       mu_star
52           dvbm  28811.355640
55           wvel  15221.987268
10  rho_greenland   9878.052951
35     glaciers_n   9431.425947
4    sd_antarctic   8809.304739
56          wvpdl   8513.566105
51     movefactor   8009.738667
15          N2O_0   7823.697573
37      anto_beta   7821.808385
41   antarctic_nu   7805.483979
             names       mu_star
52            dvbm  25610.843217
55            wvel  11557.001703
40    antarctic_mu   8357.509203
8   rho_ocean_heat   7584.727015
3     sd_g

In [122]:
dfSi[scen] = pd.DataFrame(Si)

In [125]:
dfSi.sort_values(by="mu_star", ascending=False)

,names,mu,mu_star,sigma,mu_star_conf
52,dvbm,50566.550939,50566.550939,13379.369614,6029.274964
55,wvel,-27459.723026,27459.723026,9775.805329,4616.206594
51,movefactor,12756.617520,16578.579536,13805.326037,4046.506520
22,antarctic_s0,-3998.754630,15603.042455,19012.708158,5337.682839
42,antarctic_precip0,3864.429685,14747.259195,17352.267298,4249.262894
4,sd_antarctic,3756.799376,14202.493012,16699.408021,3809.288186
45,antarctic_runoff_height0,1031.551722,14102.321853,17989.425363,4654.714525
14,CO2_0,-1937.113675,13713.303434,15980.869009,3697.909218
56,wvpdl,8756.728467,13552.561337,14754.370895,4325.696526
54,vslmult,-1050.832336,13096.740555,16878.010574,4644.893122


In [113]:
problem = {
    'num_vars': 3,
    'names': ['x1', 'x2', 'x3'],
    'bounds': [[-3, 3],
               [-3, 3],
               [-3, 3]]
}

In [43]:
X = SALib.sample.morris.sample(problem, N=1000)
Y = Ishigami.evaluate(X)
Si = SALib.analyze.morris.analyze(problem, X, Y, print_to_console=True)

          mu   mu_star     sigma  mu_star_conf
x1  7.587568  7.587568  5.898071  3.811969e-01
x2 -0.216770  7.225665  7.226027  6.982945e-15
x3  0.165075  5.962780  7.296156  2.689885e-01


In [114]:
N = 999
x1 = np.random.uniform(low=-3, high=3, size=N)
x2 = np.random.uniform(low=-3, high=3, size=N)
x3 = np.random.uniform(low=-3, high=3, size=N)
X = np.zeros((N,3))
X[:,0] = x1; X[:,1] = x2; X[:,2] = x3
Y = Ishigami.evaluate(X)
Si = SALib.analyze.morris.analyze(problem, X, Y, print_to_console=True)

ValueError: cannot reshape array of size 999 into shape (249,4)

In [40]:
x1

array([ 0.07283961,  1.51859811, -1.31975127])

              mu   mu_star     sigma  mu_star_conf
x1  9.875039e+00  9.875039  5.695639  3.025162e+00
x2  1.776357e-16  7.225665  7.616520  3.499135e-15
x3 -2.696906e+00  5.055125  6.256762  2.680622e+00


In [29]:
Si

{'names': ['x1', 'x2', 'x3'],
 'mu': array([ 9.87503948e+00,  1.77635684e-16, -2.69690640e+00]),
 'mu_star': masked_array(data=[9.875039478321025, 7.225664896786919,
                    5.055124785108947],
              mask=[False, False, False],
        fill_value=1e+20),
 'sigma': array([5.69563851, 7.61651956, 6.25676242]),
 'mu_star_conf': masked_array(data=[3.243326631683749, 3.4991350634940874e-15,
                    2.689858545453168],
              mask=[False, False, False],
        fill_value=1e+20)}